In [1]:
import sys, time, wmi, psutil
SYSTEM_INFO = wmi.WMI().Win32_OperatingSystem()[0]
"system: {0}, {1}, {2}".format(SYSTEM_INFO.Caption, SYSTEM_INFO.BuildNumber, SYSTEM_INFO.OSArchitecture) 
"memory: {}G".format(round(psutil.virtual_memory().total / 1024**3, 2))
"cpu: {}".format(psutil.cpu_count())
"python: {}".format(sys.version)
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'system: Microsoft Windows 10 教育版, 18363, 64 位'

'memory: 15.86G'

'cpu: 4'

'python: 3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]'

'2020-10-04 16:09:54'

- **@author**: run_walker
- **@references**:
    1. [Python: 浅淡Python中的属性(property)](https://www.cnblogs.com/crwy/p/6852347.html)
    2. [官方文档 > 内置函数 > property](https://docs.python.org/zh-cn/3.7/library/functions.html?highlight=property#property)

<div class="alert alert-block alert-warning">
    <i class="fa fa-sticky-note" aria-hidden="true"><b> Note:</b></i>
    python2和python3中的类并不一样。python3中默认的类支持全属性，所以不必再显示继承于object类。
</div>

# 实例属性的访问
1. 正常命名：普通的属性，可以被访问修改
2. 单下划线开头：习惯将其约定为私有属性，但仍然可以照常访问修改
3. 双下划线开头：有一定约束力的私有属性，不能直接访问，但可以修改；除此外也有办法访问

In [9]:
class A:
    def __init__(self):
        self.x = 1
        self._x = 1
        self.__x = 1

In [10]:
a = A()

In [11]:
a.x
a.x = 2
a.x

1

2

In [12]:
a._x
a._x = 2
a._x

1

2

In [15]:
try:
    a.__x
except Exception as e:
    print(e)

'A' object has no attribute '__x'


In [16]:
a.__dict__

{'x': 2, '_x': 2, '_A__x': 1}

In [19]:
a._A__x  # 还是可以访问

1

In [20]:
a.__x = 2  # 由于在命名空间中直接设定为了_A__x，所以相当于添加了一个新的属性__x
a.__dict__

{'x': 2, '_x': 2, '_A__x': 1, '__x': 2}

# property

In [21]:
?property

Init signature: property(fget=None, fset=None, fdel=None, doc=None)
Docstring:     
Property attribute.

  fget
    function to be used for getting an attribute value
  fset
    function to be used for setting an attribute value
  fdel
    function to be used for del'ing an attribute
  doc
    docstring

Typical use is to define a managed attribute x:

class C(object):
    def getx(self): return self._x
    def setx(self, value): self._x = value
    def delx(self): del self._x
    x = property(getx, setx, delx, "I'm the 'x' property.")

Decorators make defining new properties or modifying existing ones easy:

class C(object):
    @property
    def x(self):
        "I am the 'x' property."
        return self._x
    @x.setter
    def x(self, value):
        self._x = value
    @x.deleter
    def x(self):
        del self._x
Type:           type
Subclasses:     abstractproperty


## 用作函数的写法
对于函数的取名没有要求，`property`返回值赋给的变量为想要的属性名。

In [27]:
class C1:
    def __init__(self):
        self._x = None

    def get_x(self):
        print('call getter')
        return self._x

    def set_x(self, value):
        print('call setter')
        self._x = value

    def del_x(self):
        print('call deleter')
        del self._x

    x = property(get_x, set_x, del_x, "I'm the 'x' property.")

对于`C1`的实例`c1`，`c1.x`将调用`get_x`，`c1.x = value`将调用`set_x`， `del c1.x`将调用`del_x`。

In [29]:
c1 = C1()

In [30]:
c1.x

call getter


In [35]:
# doc string
?c1.x  

Type:        property
String form: <property object at 0x000001E9040123B8>
Docstring:   I'm the 'x' property.


In [36]:
c1.x = 10

call setter


In [37]:
c1.x

call getter


10

## 当装饰器的写法（推荐）
注意装饰器和函数的命名，三个函数的命名都得是想要的属性名，`property`装饰的为`getter`函数，其函数文档为属性的文档字符串。

In [39]:
class C2:
    def __init__(self):
        self._x = None
    
    @property
    def x(self):
        """I'm the 'x' property."""
        print('call getter')
        return self._x
    
    @x.setter
    def x(self, value):
        print('call setter')
        self._x = value
        
    @x.deleter
    def x(self):
        print('call deleter')
        del self._x

In [40]:
c2 = C2()

In [41]:
c2.x

call getter


In [42]:
?c2.x

Type:        property
String form: <property object at 0x000001E904010048>
Docstring:   I'm the 'x' property.


In [43]:
c2.x = 10

call setter


In [44]:
c2.x

call getter


10

## 用途
通过这种方式设置属性，通常是想要在`setter`函数中添加对该属性的约束，通过`temperature`进行修改时会调用该函数，受到约束的制约，但是仍然能通过`_temperature`不受约束地对底层值访问和修改，这是python的语言特性：自由。

In [45]:
class Celsius:
    '''摄氏温度'''
    
    def __init__(self, temperature=0):
        self._temperature = temperature

    @property
    def temperature(self):
        print("call getter")
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        print("call setter")
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        self._temperature = value

    def to_fahrenheit1(self):
        return (self._temperature * 1.8) + 32
    
    def to_fahrenheit2(self):
        return (self.temperature * 1.8) + 32

In [46]:
c = Celsius()

In [47]:
c.__dict__

{'_temperature': 0}

`_temperature`和`temperature`同时存在，前者是直接访问`_temperature`，后者是调用`temperature()`，既然采用了这种写法，还是推荐使用后者。

In [50]:
c._temperature  # 不推荐

0

In [51]:
c.temperature  # 推荐

call getter


0

In [52]:
c.temperature = 200

call setter


In [53]:
try:
    c.temperature = -300
except Exception as e:
    print(e)

call setter
Temperature below -273 is not possible


In [55]:
c._temperature = -300  # 

In [56]:
c.temperature

call getter


-300